<a href="https://colab.research.google.com/github/MohammadNasrIsfahany/retrieval-information/blob/main/phase3/notebook/retrieval_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Jun 26 03:57:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install required packages

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install pyyaml==5.4.1

     |████████████████████████████████| 4.4 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 101 kB 11.6 MB/s 
     |████████████████████████████████| 6.6 MB 33.9 MB/s 
     |████████████████████████████████| 316 kB 4.6 MB/s 
     |████████████████████████████████| 1.4 MB 45.3 MB/s 
     |████████████████████████████████| 233 kB 61.8 MB/s 
     |████████████████████████████████| 175 kB 5.1 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 235 kB 46.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 5.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [61]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections
import os

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# !unzip "/content/gdrive/MyDrive/information retrieval/dataset.zip"
!unzip "/content/gdrive/MyDrive/information retrieval/dataset.zip" > /dev/null

In [6]:
# df = pd.DataFrame(columns = ['Beinolmelal', 'Eghtesadi', 'Ejtemaee', 'ElmiVaDanishgahi', 'FarhangHonarVarResane', 'Siasi', 'Varzeshi'])
df = pd.DataFrame(columns = ['text', 'label'])

In [7]:
# create dataframe from text files

for root, dirs, files in os.walk("./"):

  if root == "./Beinolmelal":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'Beinolmelal'}, ignore_index=True)

  elif root == "./Eghtesadi":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'Eghtesadi'}, ignore_index=True)

  elif root == "./Ejtemaee":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'Ejtemaee'}, ignore_index=True)
      
  elif root == "./ElmiVaDaneshghai":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'ElmiVaDaneshgahi'}, ignore_index=True)
      
  elif root == "./FarhangHonarVaResane":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'FarhangHonarVaResane'}, ignore_index=True)

  elif root == "./Siasi":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'Siasi'}, ignore_index=True)

  elif root == "./Varzeshi":
    for file in files:
      with open(f"{root}/{file}", 'rb') as my_file:
        text_file = my_file.readlines()[0].decode(errors='replace')
        df = df.append({'text': text_file, 'label':'Varzeshi'}, ignore_index=True)

In [8]:
df

,text,label
0,خرید اقلام خوراکی و حتی چیزهای دیگر ان هم از ف...,ElmiVaDaneshgahi
1,وی با بیان اینکه بحث جوامع دور از وطن بحثی بسی...,ElmiVaDaneshgahi
2,به گزارش خبرنگار دانشگاه خبرگزاری دانشجو، ثبت ...,ElmiVaDaneshgahi
3,به گزارش خبرنگار خبرگزاری حوزه، حجت الاسلام وا...,ElmiVaDaneshgahi
4,سه شنبه 7 دی 1400 _ 11:25 شناسه خبر: 51641064 ...,ElmiVaDaneshgahi
...,...,...
29867,معاون غذا و داروی دانشکده علوم پزشکی مراغه گفت...,Ejtemaee
29868,به گزارش برنا؛ در میان اخبار ناخوشایند کرونا ک...,Ejtemaee
29869,نادر طالب زاده در گفت وگو با خبرنگار ایران انل...,Ejtemaee
29870,سعید معین مدیرکل مبارزه با افات عمومی و همگانی...,Ejtemaee


In [9]:
df.label.value_counts()

Varzeshi                5001
Beinolmelal             4958
FarhangHonarVaResane    4000
Ejtemaee                4000
Eghtesadi               3994
ElmiVaDaneshgahi        3982
Siasi                   3937
Name: label, dtype: int64

In [10]:
# calculate the length of comments based on their words
df['comment_len_by_words'] = df['text'].apply(lambda t: len(hazm.word_tokenize(t)))

In [11]:
min_max_len = df["comment_len_by_words"].min(), df["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 1 	Max: 4550


In [12]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

data_gl_than(df, 256, 3)

Texts with word length of greater than 3 and less than 256 includes 94.29% of the whole!


In [13]:
minlim, maxlim = 3, 256

In [14]:
# remove comments with the length of fewer than three words
df['comment_len_by_words'] = df['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
df = df.dropna(subset=['comment_len_by_words'])
df = df.reset_index(drop=True)

In [15]:
df

,text,label,comment_len_by_words
0,خرید اقلام خوراکی و حتی چیزهای دیگر ان هم از ف...,ElmiVaDaneshgahi,123.0
1,وی با بیان اینکه بحث جوامع دور از وطن بحثی بسی...,ElmiVaDaneshgahi,47.0
2,به گزارش خبرنگار دانشگاه خبرگزاری دانشجو، ثبت ...,ElmiVaDaneshgahi,50.0
3,به گزارش خبرنگار خبرگزاری حوزه، حجت الاسلام وا...,ElmiVaDaneshgahi,96.0
4,سه شنبه 7 دی 1400 _ 11:25 شناسه خبر: 51641064 ...,ElmiVaDaneshgahi,14.0
...,...,...,...
28162,معاون غذا و داروی دانشکده علوم پزشکی مراغه گفت...,Ejtemaee,44.0
28163,به گزارش برنا؛ در میان اخبار ناخوشایند کرونا ک...,Ejtemaee,53.0
28164,نادر طالب زاده در گفت وگو با خبرنگار ایران انل...,Ejtemaee,83.0
28165,سعید معین مدیرکل مبارزه با افات عمومی و همگانی...,Ejtemaee,42.0


In [16]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [17]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [18]:
# cleaning comments
df['cleaned_comment'] = df['text'].apply(cleaning)


# calculate the length of comments based on their words
df['cleaned_comment_len_by_words'] = df['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

df

,text,label,comment_len_by_words,cleaned_comment,cleaned_comment_len_by_words
0,خرید اقلام خوراکی و حتی چیزهای دیگر ان هم از ف...,ElmiVaDaneshgahi,123.0,خرید اقلام خوراکی و حتی چیزهای دیگر ان هم از ف...,118
1,وی با بیان اینکه بحث جوامع دور از وطن بحثی بسی...,ElmiVaDaneshgahi,47.0,وی با بیان اینکه بحث جوامع دور از وطن بحثی بسی...,46
2,به گزارش خبرنگار دانشگاه خبرگزاری دانشجو، ثبت ...,ElmiVaDaneshgahi,50.0,به گزارش خبرنگار دانشگاه خبرگزاری دانشجو، ثبت ...,48
3,به گزارش خبرنگار خبرگزاری حوزه، حجت الاسلام وا...,ElmiVaDaneshgahi,96.0,به گزارش خبرنگار خبرگزاری حوزه، حجت الاسلام وا...,94
4,سه شنبه 7 دی 1400 _ 11:25 شناسه خبر: 51641064 ...,ElmiVaDaneshgahi,14.0,سه شنبه ۷ دی ۱۴۰۰ _ ۱۱:۲۵ شناسه خبر: ۵۱۶۴۱۰۶۴ ...,14
...,...,...,...,...,...
28162,معاون غذا و داروی دانشکده علوم پزشکی مراغه گفت...,Ejtemaee,44.0,معاون غذا و داروی دانشکده علوم پزشکی مراغه گفت...,44
28163,به گزارش برنا؛ در میان اخبار ناخوشایند کرونا ک...,Ejtemaee,53.0,به گزارش برنا؛ در میان اخبار ناخوشایند کرونا ک...,50
28164,نادر طالب زاده در گفت وگو با خبرنگار ایران انل...,Ejtemaee,83.0,نادر طالب زاده در گفت وگو با خبرنگار ایران انل...,83
28165,سعید معین مدیرکل مبارزه با افات عمومی و همگانی...,Ejtemaee,42.0,سعید معین مدیرکل مبارزه با افات عمومی و همگانی...,42


In [19]:
fig = go.Figure()

groupby_label = df.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [20]:
train, test = train_test_split(df, test_size=0.1, random_state=1, stratify=df['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['text'].values.tolist(), train['label'].values.tolist()
x_valid, y_valid = valid['text'].values.tolist(), valid['label'].values.tolist()
x_test, y_test = test['text'].values.tolist(), test['label'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)


(22815, 5)
(2535, 5)
(2817, 5)


In [21]:
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

In [22]:
labels = list(sorted(df['label'].unique()))

label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'Beinolmelal': 0, 'Eghtesadi': 1, 'Ejtemaee': 2, 'ElmiVaDaneshgahi': 3, 'FarhangHonarVaResane': 4, 'Siasi': 5, 'Varzeshi': 6}
id2label: {0: 'Beinolmelal', 1: 'Eghtesadi', 2: 'Ejtemaee', 3: 'ElmiVaDaneshgahi', 4: 'FarhangHonarVaResane', 5: 'Siasi', 6: 'Varzeshi'}


In [46]:
# general config
MAX_LEN = 256
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 5
EEVERY_EPOCH = 500
LEARNING_RATE = 2e-6
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/gdrive/MyDrive/information retrieval/output/tensor_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [47]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })
config.num_labels = 7

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Beinolmelal",
    "1": "Eghtesadi",
    "2": "Ejtemaee",
    "3": "ElmiVaDaneshgahi",
    "4": "FarhangHonarVaResane",
    "5": "Siasi",
    "6": "Varzeshi"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Beinolmelal": 0,
    "Eghtesadi": 1,
    "Ejtemaee": 2,
    "ElmiVaDaneshgahi": 3,
    "FarhangHonarVaResane": 4,
    "Siasi": 5,
    "Varzeshi": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



In [48]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = _y
        
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    
    features = glue_convert_examples_to_features(
        examples, 
        tokenizer, 
        maxlen, 
        output_mode=output_mode, 
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [49]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=256)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_valid, y_valid, maxlen=256)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, y_test, maxlen=256)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, y_test, maxlen=256, is_tf_dataset=False)

  0%|          | 0/22815 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/data/processors/glue.py:66: FutureWarning:

This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py



  0%|          | 0/22815 [00:00<?, ?it/s]

  0%|          | 0/2535 [00:00<?, ?it/s]

  0%|          | 0/2535 [00:00<?, ?it/s]

  0%|          | 0/2817 [00:00<?, ?it/s]

  0%|          | 0/2817 [00:00<?, ?it/s]

  0%|          | 0/2817 [00:00<?, ?it/s]

  0%|          | 0/2817 [00:00<?, ?it/s]

In [50]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')

     input_ids: [    2 18654 50999  6836  2017 64803  4040  3264  2011 64123  2812  2788
  1348 33858 43973  2008  1348 14789 28311  2020  3366  9406 11057 19262
 57710  2011  1012     4     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     

In [51]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)

    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)

    return dataset

In [52]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
valid_dataset = get_training_dataset(valid_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
valid_steps = len(valid_examples) // VALID_BATCH_SIZE

train_steps, valid_steps

(1425, 158)

In [53]:
def build_model(model_name, config, learning_rate=2e-6):
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [54]:
model = build_model(MODEL_NAME_OR_PATH, config, learning_rate=LEARNING_RATE)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

Epoch 1/5
1425/1425 [==============================] - 1264s 887ms/step - loss: 1.0530 - accuracy: 0.6804 - val_loss: 0.6890 - val_accuracy: 0.7860
Epoch 2/5
1425/1425 [==============================] - 1259s 884ms/step - loss: 0.6386 - accuracy: 0.7958 - val_loss: 0.6466 - val_accuracy: 0.7971
Epoch 3/5
1425/1425 [==============================] - 1259s 884ms/step - loss: 0.5591 - accuracy: 0.8218 - val_loss: 0.5807 - val_accuracy: 0.8161
Epoch 4/5
1425/1425 [==============================] - 1259s 884ms/step - loss: 0.4977 - accuracy: 0.8401 - val_loss: 0.5887 - val_accuracy: 0.8062
Epoch 5/5
1425/1425 [==============================] - 1259s 884ms/step - loss: 0.4523 - accuracy: 0.8539 - val_loss: 0.5931 - val_accuracy: 0.8097
FINAL ACCURACY MEAN:  0.8030063271522522


In [74]:
# predictions = model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()
print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}\n')
print(f'accuracy: {accuracy_score(ytest, ypred)}\n')
print(f'confusion matrix: \n{confusion_matrix(ytest, ypred)}')


                      precision    recall  f1-score   support

         Beinolmelal       0.81      0.83      0.82       464
           Eghtesadi       0.79      0.76      0.78       367
            Ejtemaee       0.81      0.71      0.75       381
    ElmiVaDaneshgahi       0.76      0.76      0.76       373
FarhangHonarVaResane       0.79      0.81      0.80       383
               Siasi       0.70      0.80      0.75       373
            Varzeshi       0.95      0.93      0.94       476

            accuracy                           0.80      2817
           macro avg       0.80      0.80      0.80      2817
        weighted avg       0.81      0.80      0.80      2817


F1: 0.8049897707396596

accuracy: 0.8047568335108272

confusion matrix: 
[[383   7  10  12   7  38   7]
 [ 12 279  14  21  10  26   5]
 [ 14  13 269  37  21  22   5]
 [ 10  20  17 284  24  16   2]
 [ 27  14   9   7 312  12   2]
 [ 18  17  13   8  17 297   3]
 [  8   3   2   5   2  13 443]]


In [ ]:
ntrain = train[['text', 'label']]
ntrain.columns = ['text', 'label']

ndev = valid[['text', 'label']]
ndev.columns = ['text', 'label']

ntest = test[['text', 'label']]
ntest.columns = ['text', 'label']


ntrain.to_csv('/content/gdrive/MyDrive/information retrieval/' + 'train.tsv', sep='\t', index=False)
ndev.to_csv('/content/gdrive/MyDrive/information retrieval/' + 'dev.tsv', sep='\t', index=False)
ntest.to_csv('/content/gdrive/MyDrive/information retrieval/' + 'test.tsv', sep='\t', index=False)

In [ ]:
!git clone https://github.com/hooshvare/parsbert.git parsbert
%cd /content/parsbert/scripts/
!rm -rf __pycache__
!ls

Cloning into 'parsbert'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 122 (delta 44), reused 83 (delta 22), pack-reused 0
Receiving objects: 100% (122/122), 1.15 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/parsbert/scripts
__init__.py  run_clf_finetuning.py  utils.py


In [ ]:
!python run_clf_finetuning.py

usage: run_clf_finetuning.py [-h] --output_mode OUTPUT_MODE
                             [--convert_to_tf [CONVERT_TO_TF]]
                             [--no_convert_to_tf] [--labels LABELS]
                             --model_name_or_path MODEL_NAME_OR_PATH
                             [--config_name CONFIG_NAME]
                             [--tokenizer_name TOKENIZER_NAME]
                             [--cache_dir CACHE_DIR] --task_name TASK_NAME
                             --data_dir DATA_DIR
                             [--max_seq_length MAX_SEQ_LENGTH]
                             [--overwrite_cache [OVERWRITE_CACHE]]
                             --output_dir OUTPUT_DIR
                             [--overwrite_output_dir [OVERWRITE_OUTPUT_DIR]]
                             [--do_train [DO_TRAIN]] [--do_eval [DO_EVAL]]
                             [--do_predict [DO_PREDICT]]
                             [--evaluation_strategy {no,steps,epoch}]
                             [--pr

In [ ]:
!mkdir -p '/content/gdrive/MyDrive/information retrieval/output'


!python run_clf_finetuning.py \
    --labels FarhangHonarVaResane,ElmiVaDaneshgahi,Beinolmelal,Ejtemaee,Varzeshi,Eghtesadi,Siasi \
    --output_mode classification \
    --task_name akhbar \
    --model_name_or_path HooshvareLab/bert-fa-base-uncased \
    --data_dir '/content/gdrive/MyDrive/information retrieval/' \
    --output_dir '/content/gdrive/MyDrive/information retrieval/output' \
    --max_seq_length 257 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --learning_rate 2e-5 \
    --num_train_epochs 3.0 \
    --logging_steps 500 \
    --save_steps 1000 \
    --do_train \
    --do_eval \
    --do_predict \
    --overwrite_output_dir \
    --logging_first_step

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=Fals